# Using Autoencoder
em construcao

In [1]:
import pandas as pd
import numpy as np
import talib as ta
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from scipy.signal import argrelextrema
import json
from bs4 import BeautifulSoup
import requests 
import urllib.request
import datetime
from datetime import timedelta, datetime
import time
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from keras import regularizers
from keras.models import Sequential, model_from_json, Model
from keras.layers import Dense, Dropout, LSTM, Activation, Input, Conv2D,MaxPooling2D, UpSampling2D, Flatten, Reshape
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import confusion_matrix
diretorio= os.getcwd()
diretorio

/Users/igoracmorais/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/Users/igoracmorais/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


'/Users/igoracmorais/igor/Wise&Trust/CPM_trading2/assets-management/Trading/Rotines/Python'

In [2]:
def macd(data,n=200,nome_coluna_datas="datetime",EMA_fast=[],EMA_slow=[]):
    for a in EMA_fast:
        for b in EMA_slow:
            data['macd'+str(a)+str(b)],data['macdsignal'+str(a)+str(b)],data['macdhist'+str(a)+str(b)]=ta.MACD(data.close,
                                                                                                               fastperiod=a,
                                                                                                               slowperiod=b,
                                                                                                               signalperiod=9)
            data['EMA'+str(b)]=ta.EMA(data.close,timeperiod=b)
        data['EMA'+str(a)]=ta.EMA(data.close,timeperiod=a)
    return data
###
def turning_point(data,coluna=[],ordem=3):
    import warnings
    warnings.filterwarnings("ignore")
    for coluna in coluna:
        data[coluna+'_min'] = data.iloc[argrelextrema(data[coluna].values, 
                                                  np.less_equal, order=ordem)[0]][coluna]
        data[coluna+'_max'] = data.iloc[argrelextrema(data[coluna].values, 
                                                  np.greater_equal, order=ordem)[0]][coluna]
        data[coluna+'_min'].fillna(0,inplace=True)
        data[coluna+'_max'].fillna(0,inplace=True)
        data["tp_"+coluna]=0
        data.loc[data[coluna+'_min']>0,"tp_"+coluna]=1
        data.loc[data[coluna+'_max']>0,"tp_"+coluna]=-1
        data[[coluna+'_min',coluna+'_max']]=np.nan
        data.drop([coluna+'_min',coluna+'_max'],axis=1,inplace=True)
    return data
###
def adx(data,time=[]):
    for adx in time:
        data["adx_"+str(adx)]=ta.ADX(data.high,data.low,data.close,timeperiod=adx)
        data["di-_"+str(adx)]=ta.MINUS_DI(data.high,data.low,data.close,timeperiod=adx)
        data["di+_"+str(adx)]=ta.PLUS_DI(data.high,data.low,data.close,timeperiod=adx)
        # sinais - trend
        data["dmi_s"+str(adx)]=np.where(data["di+_"+str(adx)]>data["di-_"+str(adx)],1,-1)
    return data
###
def apo(data,EMA_fast=[],EMA_slow=[]):
    for f in EMA_fast:
        for s in EMA_slow:
            data['apo_'+str(f)+'_'+str(s)]=ta.APO(data.close,fastperiod=f, slowperiod=s, matype=0)
    return data
###
def aroon(data,time=[]):
    for aroon in time:
        data['aroon_down'+str(aroon)], data['aroon_up'+str(aroon)] = ta.AROON(data.high,data.low,timeperiod=aroon)
        # sinais - trend
        data['aroon_s'+str(aroon)]=np.where(data['aroon_down'+str(aroon)]<data['aroon_up'+str(aroon)],1,-1)
    return data
###
def bop(data,time=[]):
    data["bop"]=ta.BOP(data.open, data.high, data.low, data.close)
    for t in time:
        data["bop_"+str(t)]=ta.EMA(data.bop,timeperiod=t)
    return data
###
def sinais_cci(data,intervalo=100,time=[]):
    for i in time:
        data["cci_"+str(i)]=ta.CCI(data.high,data.low,data.close,timeperiod=i)
        data["cci_s"+str(i)]=0
        data.loc[(data["cci_"+str(i)].shift(1)<intervalo)&(data["cci_"+str(i)]>intervalo),"cci_s"+str(i)]=1
        data.loc[(data["cci_"+str(i)].shift(1)>-intervalo)&(data["cci_"+str(i)]<-intervalo),"cci_s"+str(i)]=-1
    return data
###
def cmo(data,overbought=50,oversold=-50,time=[]):
    for cmo in time:
        data["cmo_"+str(cmo)]=ta.CMO(data.close,timeperiod=cmo)
        # sinais
        data["cmo_s"+str(cmo)]=0
        data.loc[(data["cmo_"+str(cmo)]>overbought),"cmo_s"+str(cmo)]=-1
        data.loc[(data["cmo_"+str(cmo)]<oversold),"cmo_s"+str(cmo)]=1
    return data
###
def mfi(data,overbought=90,oversold=20,time=[]):
    for mfii in time:
        data["mfi_"+str(mfii)]=ta.MFI(data.high,data.low,data.close,data.volume,timeperiod=mfii)
        data["mfi_s"+str(mfii)]=0
        data.loc[(data["mfi_"+str(mfii)] >= overbought),"mfi_s"+str(mfii)]=-1
        data.loc[(data["mfi_"+str(mfii)] <= oversold),"mfi_s"+str(mfii)]=1
    return data
###
def mom(data,time=[]):
    for i in time:
        data["mom_"+str(i)]=ta.MOM(data.close,timeperiod=i)
    return data
###
def ppo(data,EMA_fast=[],EMA_slow=[]):
    for pf in EMA_fast:
        for ps in EMA_slow:
            data["ppo_"+str(pf)+'_'+str(ps)]=ta.PPO(data.close,fastperiod=pf,slowperiod=ps,matype=0)
    return data
###
def sinais_rsi(data,time=[]):
    for rsii in time:
        data["rsi_"+str(rsii)]=ta.RSI(data.close,timeperiod=rsii)
        data["rsi_s"+str(rsii)]=0
        data.loc[(data["rsi_"+str(rsii)] >= 60),"rsi_s"+str(rsii)]=-1
        data.loc[(data["rsi_"+str(rsii)] <= 40),"rsi_s"+str(rsii)]=1
    return data
###
def stoch(data,EMA_fast=[],EMA_slow=[]):
    for ft in EMA_fast:
        for st in EMA_slow:
            data["slowk_"+str(ft)+"_"+str(st)],data["slowd_"+str(ft)+"_"+str(st)]= ta.STOCH(data.high,
                                                                                            data.low,
                                                                                            data.close,
                                                                                           fastk_period=ft,
                                                                                           slowk_period=st,
                                                                                           slowk_matype=0,
                                                                                           slowd_period=3,
                                                                                           slowd_matype=0)
            # signal turning point
            data=turning_point(data,coluna=["slowk_"+str(ft)+"_"+str(st)])
            #
            data["fastk_"+str(ft)+"_"+str(st)],data["fastd_"+str(ft)+"_"+str(st)]= ta.STOCHF(data.high,
                                                                                            data.low,
                                                                                            data.close,
                                                                                            fastk_period=ft,
                                                                                            fastd_period=3,
                                                                                            fastd_matype=0)
            # signal fastk
            data["fastk_"+str(ft)+"_"+str(st)+"_"+"s"]=0
            data.loc[(data["fastk_"+str(ft)+"_"+str(st)] > 50),"fastk_"+str(ft)+"_"+str(st)+"_"+"s"]=1
            data.loc[(data["fastk_"+str(ft)+"_"+str(st)] <= 50),"fastk_"+str(ft)+"_"+str(st)+"_"+"s"]=-1
    return data
###
def william(data,time=[]):
    for w in time:
        data["will_"+str(w)]=ta.WILLR(data.high, data.low, data.close, timeperiod=w)
        # sinal over
        data["will_"+str(w)+"_"+"s"]=0
        data.loc[(data["will_"+str(w)+"_"+"s"] > -50),"will_"+str(w)+"_"+"s"]=1
        data.loc[(data["will_"+str(w)+"_"+"s"] <= -50),"will_"+str(w)+"_"+"s"]=-1
        # sinal turning point
        data=turning_point(data,coluna=["will_"+str(w)])
    return data

In [3]:
def indicadores(data):
    import warnings
    warnings.filterwarnings("ignore")
    df=macd(data,EMA_fast=[6],EMA_slow=[22])
    df=turning_point(df,coluna=["macd622","macdsignal622","macdhist622"])
    df=adx(df,time=[6,18,30])
    df=apo(df,EMA_fast=[6],EMA_slow=[22])
    df=sinal_apo(df,coluna=["apo_6_22"])
    df=aroon(df,time=[6,18,30])
    df=bop(df,time=[6,18,30])
    df=sinais_cci(df,intervalo=100,time=[6,18,30])
    df=cmo(df,overbought=50,oversold=-50,time=[6,18,30])
    df=mfi(df,overbought=90,oversold=20,time=[6,18,30])
    df=mom(df,time=[6,18,30])
    df=ppo(df,EMA_fast=[6],EMA_slow=[30])
    df=sinais_rsi(df,time=[6,18,30])
    df=stoch(df,EMA_fast=[6],EMA_slow=[22])
    df=william(df,time=[6,18,30])
    df=df.dropna(axis=0)
    return df

In [4]:
def sinal_apo(data,coluna=[]):
    for k in coluna:
        data["r_apo_"+str(k)]=(data[k]/data[k].shift(1)-1)*100
        data["r_apo_s"+str(k)]=0
        data.loc[(data["r_apo_"+str(k)] > 0),"r_apo_s"+str(k)]=1
        data.loc[(data["r_apo_"+str(k)] < 0),"r_apo_s"+str(k)]=-1
    return data

In [5]:
def var_y(df,train=0.8):
    from sklearn.preprocessing import StandardScaler, QuantileTransformer
    scaler2=QuantileTransformer(output_distribution='uniform')
    df["y"]=np.where(df["close"].shift(-1)>df["close"],1,0)
    x=df.iloc[:,:-1]
    y=pd.DataFrame(df.iloc[:,-1])
    # train and test
    if train==1:
        x=scaler2.fit_transform(x)
        return df,x,y
    else:
        split = int(len(df)*train)
        x_train, x_test, y_train, y_test = x[:split],x[split:],y[:split],y[split:]
        # scaling
        x_train=scaler2.fit_transform(x_train)
        x_test=scaler2.fit_transform(x_test)
    return df,x,y, x_train, x_test, y_train, y_test

In [6]:
def reshape(df,step=1):
    df=np.array(df)
    df2=np.reshape(df,(df.shape[0],step,df.shape[1]))
    return df2

## Exemplo

In [7]:
data=pd.read_csv(diretorio+"/btc.csv",sep=";",index_col="datetime")
data2=data[:-20:]
df=indicadores(data2)
df,x,y= var_y(df,train=1)
x.shape, y.shape

((1922, 84), (1922, 1))

In [10]:
input_dim=x.shape[1]
encoding_dim=3   #número de caracteristicas que queremos extrair
autoencoder=Sequential()
autoencoder.add(Dense(70,input_shape=(input_dim,),activation="relu",activity_regularizer=regularizers.l2(0)))
autoencoder.add(Dropout(0.2))
autoencoder.add(Dense(50,activation="relu"))
autoencoder.add(Dense(30,activation="relu"))
autoencoder.add(Dropout(0.2))
autoencoder.add(Dense(15,activation="relu"))
autoencoder.add(Dense(encoding_dim, activation="relu",name='reduction'))
#
autoencoder.add(Dense(15,activation="relu"))
autoencoder.add(Dense(30,activation="relu"))
autoencoder.add(Dense(50,activation="relu"))
autoencoder.add(Dense(70,activation="relu"))
autoencoder.add(Dense(input_dim,activation="sigmoid"))
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 70)                5950      
_________________________________________________________________
dropout_3 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                3550      
_________________________________________________________________
dense_12 (Dense)             (None, 30)                1530      
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 15)                465       
_________________________________________________________________
reduction (Dense)            (None, 3)                 48        
__________

In [11]:
# vejamos como é a parte do encoder
inputs=Input(shape=(input_dim,))
encoder_layer1=autoencoder.layers[0]
encoder_layer2=autoencoder.layers[1]
encoder_layer3=autoencoder.layers[2]
encoder_layer4=autoencoder.layers[3]
encoder_layer5=autoencoder.layers[4]
encoder_layer6=autoencoder.layers[5]
encoder_layer7=autoencoder.layers[6]
encoder=Model(inputs,encoder_layer7(encoder_layer6(encoder_layer5(encoder_layer4(encoder_layer3(encoder_layer2(encoder_layer1(inputs))))))))
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 84)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 70)                5950      
_________________________________________________________________
dropout_3 (Dropout)          (None, 70)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 50)                3550      
_________________________________________________________________
dense_12 (Dense)             (None, 30)                1530      
_________________________________________________________________
dropout_4 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 15)                465       
__________

In [12]:
# vamos treinar o modelo. Veja que nao usamos a variavel y
autoencoder.compile(optimizer="adam",loss="mean_squared_error")
autoencoder.fit(x,x,epochs=400,batch_size=100,shuffle=False)

Epoch 1/400
1922/1922 [==============================] - 1s 346us/step - loss: 0.1074
Epoch 2/400
1922/1922 [==============================] - 0s 33us/step - loss: 0.0960
Epoch 3/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0762
Epoch 4/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0625
Epoch 5/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0569
Epoch 6/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0572
Epoch 7/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0546
Epoch 8/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0504
Epoch 9/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0496
Epoch 10/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0492
Epoch 11/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0488
Epoch 12/400
1922/1922 [==============================] - 0s 3

1922/1922 [==============================] - 0s 29us/step - loss: 0.0337
Epoch 97/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0336
Epoch 98/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0337
Epoch 99/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0336
Epoch 100/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0336
Epoch 101/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0337
Epoch 102/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0338
Epoch 103/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0338
Epoch 104/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0335
Epoch 105/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0334
Epoch 106/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0333
Epoch 107/400
1922/1922 [==============================] - 0

1922/1922 [==============================] - 0s 28us/step - loss: 0.0300
Epoch 191/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0301
Epoch 192/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0299
Epoch 193/400
1922/1922 [==============================] - 0s 29us/step - loss: 0.0297
Epoch 194/400
1922/1922 [==============================] - 0s 29us/step - loss: 0.0295
Epoch 195/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0295
Epoch 196/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0294
Epoch 197/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0294
Epoch 198/400
1922/1922 [==============================] - 0s 32us/step - loss: 0.0296
Epoch 199/400
1922/1922 [==============================] - 0s 31us/step - loss: 0.0293
Epoch 200/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0293
Epoch 201/400
1922/1922 [==============================] 

1922/1922 [==============================] - 0s 27us/step - loss: 0.0261
Epoch 285/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0261
Epoch 286/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0263
Epoch 287/400
1922/1922 [==============================] - 0s 25us/step - loss: 0.0260
Epoch 288/400
1922/1922 [==============================] - 0s 25us/step - loss: 0.0258
Epoch 289/400
1922/1922 [==============================] - 0s 26us/step - loss: 0.0263
Epoch 290/400
1922/1922 [==============================] - 0s 26us/step - loss: 0.0261
Epoch 291/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0262
Epoch 292/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0261
Epoch 293/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0261
Epoch 294/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0267
Epoch 295/400
1922/1922 [==============================] 

1922/1922 [==============================] - 0s 30us/step - loss: 0.0238
Epoch 379/400
1922/1922 [==============================] - 0s 26us/step - loss: 0.0238
Epoch 380/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0244
Epoch 381/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0240
Epoch 382/400
1922/1922 [==============================] - 0s 27us/step - loss: 0.0244
Epoch 383/400
1922/1922 [==============================] - 0s 26us/step - loss: 0.0243
Epoch 384/400
1922/1922 [==============================] - 0s 28us/step - loss: 0.0240
Epoch 385/400
1922/1922 [==============================] - 0s 26us/step - loss: 0.0239
Epoch 386/400
1922/1922 [==============================] - 0s 30us/step - loss: 0.0238
Epoch 387/400
1922/1922 [==============================] - 0s 26us/step - loss: 0.0239
Epoch 388/400
1922/1922 [==============================] - 0s 26us/step - loss: 0.0238
Epoch 389/400
1922/1922 [==============================] 

In [25]:
encoded_ts=pd.DataFrame(encoder.predict(x))
print(encoded_ts.shape)
encoded_ts.head(3)

(1922, 3)


,0,1,2
0,1.325963,1.074327,1.699930
1,1.336485,1.065976,1.777328
2,1.998548,0.413600,3.345315
